# Loading GA Sentinel-1 Backscatter from a STAC endpoint

## Imports

In [ ]:
from pystac_client import Client
from odc.stac import load, configure_s3_access
from odc.geo import BoundingBox
import numpy as np

## Connect to Digital Earth development STAC endpoint

In [ ]:
catalog = "https://explorer.dev.dea.ga.gov.au/stac"

stac_client = Client.open(catalog)

configure_s3_access(
    cloud_defaults=True, 
    aws_unsigned=True,
)

## View available collections

In [ ]:
results = stac_client.collection_search(q="ga_s1*")

collections = [result.id for result in results.collection_list()]

collections

## Searching and Loading

This section provides examples for both Australia and Antarctica, as there are some differences between the two:

| Property | Australia | Antarctica |
| --- | ----------- | ----------- |
| Primary capture mode | VV+VH | HH |
| Recommended CRS (metres) | UTM or EPSG:3577 | EPSG:3031 |
| Group by method | Solar day | Scene ID |


### Australia

Here, we use a bounding box over Lake Sorell and Lake Crescent in Tasmania, and a date range of June 2020 through mid-July 2020.

In [ ]:
aus_bbox = BoundingBox(
    left=147.07251,
    bottom=-42.22120,
    right=147.24274,
    top=-42.03035,
    crs="EPSG:4326"
)

aus_start_date = "2020-06-01"
aus_end_date = "2020-07-15"

The first step is to search for all observations that match these criteria, referred to as items in STAC. 

In [ ]:
collections_query = collections
aus_date_query = f"{aus_start_date}/{aus_end_date}"
aus_bbox_query = aus_bbox.bbox

aus_items = stac_client.search(
    collections=collections_query,
    datetime=aus_date_query,
    bbox=aus_bbox_query
).item_collection()

print(f"Found {len(aus_items)} items")

When working with GA's Sentinel-1 products, each item corresponds to a single burst, which is a subset of a scene.
The items contain the full STAC metadata, so information can be extracted from the properties.
For example, below demonstrates the number of unique scenes that the 33 bursts come from:

In [ ]:
unique_scenes = set([item.properties["nrb:source_id"][0] for item in aus_items])

print(f"The identified items come from {len(unique_scenes)} unique scenes")

Once the items have been identified, we use `odc-stac` to load them. 
With the `odc-stac` `load` command, you can specify:

* `crs`: the coordinate reference system (CRS) to project loaded data to. If not specified, the data's native CRS will be used.
* `resolution`: the resolution to load the data at, in the same units as the chosen CRS. If not specified, the data's native resolution will be used.
* `intersects`: a bounding box to clip the loaded data to. If not specified, the whole item will be loaded.
* `bands`: the measurements to load from the data (e.g. VV). If not specified, all measurements will be loaded.
* `groupby`: how to group loaded data. For Australia, the value `solar_day` will ensure all bursts captured on the same day are grouped together under one time-stamp.
* `chunks={}`: request that the data be lazily loaded - an xarray showing the expected dimensions and measurements will be returned. Data will be computed when used. If not specified, data will be loaded into memory. 

> **Note:** When selecting a CRS for data over Australia, we recommend "utm" or "EPSG:3577" to get data back in a coordinate system that uses metres. "utm" will return the UTM projection that is most appropriate given the bounding box of the data. If loading data over large portions of Australia, Australian Albers (EPSG:3577) or another CRS may be more appropriate.

In [ ]:
# Lazy load our filtered data
aus_ds = load(
    aus_items,
    crs="utm",
    resolution=20,
    intersects=aus_bbox.boundary(),
    bands=["VV", "VH", "mask"],
    groupby="solar_day",
    chunks={},
)

aus_ds

### Antarctica

Here, we use a bounding box over Canada Glacier in eastern Antarctica, with a date range of June 2018 through July 2018.

In [ ]:
ant_bbox = BoundingBox(
    left=162.8555,
    bottom=-77.6376,
    right=163.0801,
    top=-77.5813,
    crs="EPSG:4326"
)

ant_start_date = "2018-06-01"
ant_end_date = "2018-07-31"

The first step is to search for all observations that match these criteria, referred to as items in STAC. 

In [ ]:
collections_query = collections
ant_date_query = f"{ant_start_date}/{ant_end_date}"
ant_bbox_query = ant_bbox.bbox

ant_items = stac_client.search(
    collections=collections_query,
    datetime=ant_date_query,
    bbox=ant_bbox_query
).item_collection()

print(f"Found {len(ant_items)} items")

When working with GA's Sentinel-1 products, each item corresponds to a single burst, which is a subset of a scene.
The items contain the full STAC metadata, so information can be extracted from the properties.
For example, below demonstrates the number of unique scenes that the 10 bursts come from:

In [ ]:
unique_scenes = set([item.properties["nrb:source_id"][0] for item in ant_items])

print(f"The identified items come from {len(unique_scenes)} unique scenes")

Once the items have been identified, we use `odc-stac` to load them. 
With the `odc-stac` `load` command, you can specify:

* `crs`: the coordinate reference system (CRS) to project loaded data to. If not specified, the data's native CRS will be used.
* `resolution`: the resolution to load the data at, in the same units as the chosen CRS. If not specified, the data's native resolution will be used.
* `intersects`: a bounding box to clip the loaded data to. If not specified, the whole item will be loaded.
* `bands`: the measurements to load from the data (e.g. HH). If not specified, all measurements will be loaded.
* `groupby`: how to group loaded data. For Antarctica, the value `nrb:source_id` will ensure all bursts captured within a scene are grouped together under one time-stamp.
* `chunks={}`: request that the data be lazily loaded - an xarray showing the expected dimensions and measurements will be returned. Data will be computed when used. If not specified, data will be loaded into memory. 

> **Note:** When selecting a CRS for data over Antarctica, we recommend "EPSG:3031", which matches the data's native projection.

In [ ]:
# Lazy load our filtered data
ant_ds = load(
    ant_items,
    crs="EPSG:3031",
    resolution=20,
    intersects=ant_bbox.boundary(),
    bands=["HH", "mask"],
    groupby="nrb:source_id",
    chunks={},
)

ant_ds

## Loading data in memory

Once you have decided you are happy with the area of interest, crs, resolution, bands, etc., you can load data into memory using xarray's `.compute` operation. 

This is valuable once you are ready to apply transformations to the data, or wish to visualise the data, as it will save needing to read the data into memory every time.

In [ ]:
aus_ds = aus_ds.compute()
ant_ds = ant_ds.compute()

## Transforming and visualising loaded data

The GA Sentinel-1 backscatter products are provided as linear gamma0 values. 
It is common to apply some transformations, such as speckle filtering, and converting from linear scale to decibels. 
Converting to other measurements conventions (e.g. sigma0 and beta0) is not currently available.
If you need these measurement conventions for your work, please contact Earth.Observation@ga.gov.au and ask to have your email sent to the Digital Earth Antarctica team.

## Visualising

To see the full timeseries for a particular band, the following plotting command can be used:

### Australia

The dark areas are the two lakes.

In [ ]:
aus_ds['VV'].plot.imshow(col="time", col_wrap=3, robust=True, cmap="Greys_r")

### Antarctica
The dark area corresponds to the glacier.
The bright segments are likely due to layover as there is significant terrain in this region.

In [ ]:
ant_ds['HH'].plot.imshow(col="time", col_wrap=3, robust=True, cmap="Greys_r")

## Masking

The GA Sentinel-1 backscatter product comes with a mask that indicates invalid pixels, along with pixels impacted by layover and shadow. 

The masks have the following values:
| Value | Property |
| --- | ----------- |
| 0 | Valid | 
| 1 | Shadow |
| 2 | Layover |
| 3 | Shadow and layover |
| 255 / NaN | Invalid |

The following code displays the masks and shows how to apply them.

### Australia

Viewing the mask for the first timestep, there is very little layover or shadow in this area. 
This is consistent with there not being much terrain in the chosen area.

In [ ]:
aus_ds["mask"].isel(time=0).plot.imshow(
    levels=[-0.5, 0.5, 1.5, 2.5, 3.5], 
    cbar_kwargs={'ticks': [0, 1, 2, 3]}
)

#### Applying the mask

In [ ]:
aus_ds["VV_masked"] = aus_ds.VV.where(aus_ds.mask==0, np.nan)
aus_ds["VV_masked"].isel(time=0).plot.imshow(robust=True)

### Antarctica

Viewing the mask for the first timestep, there is significant layover in this area, and a little shadow. 
This is consistent with there being signifiant terrain in the chosen area.

In [ ]:
ant_ds["mask"].isel(time=0).plot.imshow(
    levels=[-0.5, 0.5, 1.5, 2.5, 3.5], 
    cbar_kwargs={'ticks': [0, 1, 2, 3]}
)

#### Applying the mask

In [ ]:
ant_ds["HH_masked"] = ant_ds.HH.where(ant_ds.mask==0, np.nan)
ant_ds["HH_masked"].isel(time=0).plot.imshow(robust=True)

### Speckle filtering

Speckle filtering aims to reduce noise present in SAR images.
One filter that is commonly applied is the Lee filter.
We have supplied a python file that contains the Lee filter definition, as well as a function to apply it to xarrays.

In [ ]:
from speckle_filters import apply_lee_filter

In [ ]:
aus_ds["VV_filtered"] = apply_lee_filter(aus_ds["VV"])

aus_ds